In [28]:
import feedparser
import re
from bs4 import BeautifulSoup
import json

In [20]:
def cleanhtml(raw_html):
    '''
    
    функция, которая убирает из текста тэги/литералы, очищает его
    '''
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return ' '.join(cleantext.split())

In [116]:
def get_news(user_id):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    chosen_news = []
    
    # запоминание пользовательских предпочтений
    try:
        categories = current['users'][user_id]['categories']
        keywords = current['users'][user_id]['keywords']
        links = [y[0] for y in [current['sources'][x] for x in categories]]
    except:
        return('ошибка данных')
    
    # анализ каждой новости из каждой ссылки
    for link in links:
        news_line = feedparser.parse(link).entries
        for news in news_line:
            content = cleanhtml(news.summary)
            if check_it(content, keywords):
                if 'document.write' in content:
                    content = content[: content.index('document.write') - 1]
                clean = {'title': news.title, 'link': news.links[0]['href'], 'content': content}
                chosen_news.append(clean)
    return chosen_news

In [26]:
test_id = "268178395"

In [117]:
get_news(test_id)

[{'title': 'Зеленский призвал решить конфликт в Донбассе дипломатическим путем',
  'link': 'https://lenta.ru/news/2019/10/28/mirumir/',
  'content': 'Разведение сил и средств в поселках Золотое и Петровское необходимо для разрешения конфликта в Донбассе, заявил президент Украины Владимир Зеленский. По его словам, Киеву необходимо попытаться восстановить мир дипломатическим путем. Зеленский отметил, что с Россией нельзя вести диалог с позиции силы.'},
 {'title': 'Раскрыты подробности разоружения украинских националистов в Донбассе',
  'link': 'https://lenta.ru/news/2019/10/28/bezoruzhnye/',
  'content': 'Украинские националисты вывезли оружие из поселка Золотое в Донбассе в сопровождении полицейских, рассказал заместитель главы Национальной полиции Украины Вадим Троян. По его словам, правоохранительные органы будут проводить в зоне конфликта «антидиверсионную работу против незаконных вооруженных формирований».'},
 {'title': 'Горбачев засомневался в желании США заключить новый договор вз

In [119]:
test_ans = [{'title': 'Зеленский призвал решить конфликт в Донбассе дипломатическим путем',
  'link': 'https://lenta.ru/news/2019/10/28/mirumir/',
  'content': 'Разведение сил и средств в поселках Золотое и Петровское необходимо для разрешения конфликта в Донбассе, заявил президент Украины Владимир Зеленский. По его словам, Киеву необходимо попытаться восстановить мир дипломатическим путем. Зеленский отметил, что с Россией нельзя вести диалог с позиции силы.'}] 

In [122]:
def reformat(text):
    return text['title'] + '\n\n' + text['content'] + '\n\n' + text['link']

In [129]:
print(reformat(test_ans[0]))

Зеленский призвал решить конфликт в Донбассе дипломатическим путем

Разведение сил и средств в поселках Золотое и Петровское необходимо для разрешения конфликта в Донбассе, заявил президент Украины Владимир Зеленский. По его словам, Киеву необходимо попытаться восстановить мир дипломатическим путем. Зеленский отметил, что с Россией нельзя вести диалог с позиции силы.

https://lenta.ru/news/2019/10/28/mirumir/


#### функция, которая проверяет наличие ключевых слов в новости

In [61]:
import string
import nltk
from nltk.corpus import stopwords
import pymorphy2

In [85]:
def check_it(text, keywords):
    morph = pymorphy2.MorphAnalyzer()
    
    keywords = set([morph.parse(word.lower())[0].normal_form for word in keywords])
    word_list = set([morph.parse(i.lower())[0].normal_form for i in nltk.word_tokenize(text) \
                     if i not in string.punctuation + '»«–' and i not in stopwords.words('russian')])
    
    return keywords & word_list != set()

In [50]:
test_news = '''
            Главный тренер «Манчестер Юнайтед» Оле-Гуннар Сульшер 
            прокомментировал ситуацию с травмой полузащитника Поля Погба. 
            «Не думаю, что мы увидим Поля до декабря. 
            Он будет отсутствовать еще некоторое время. Ему нужно время, 
            чтобы полностью восстановиться. Вероятно, наступит декабрь, 
            прежде чем мы сможем увидеть его на поле», – сказал Сульшер. 
            Француз остается вне игры с 30 сентября из-за травмы. 
            Ранее сообщалось, что Погба вернется в строй в ноябре.
            '''

In [93]:
check_it(test_news, ['авбу', 'погба'])

True

In [126]:
str(test_ans[0]['title'] + '\n\n' + test_ans[0]['content'] + '\n\n' + test_ans[0]['link'])

'Зеленский призвал решить конфликт в Донбассе дипломатическим путем\n\nРазведение сил и средств в поселках Золотое и Петровское необходимо для разрешения конфликта в Донбассе, заявил президент Украины Владимир Зеленский. По его словам, Киеву необходимо попытаться восстановить мир дипломатическим путем. Зеленский отметил, что с Россией нельзя вести диалог с позиции силы.\n\nhttps://lenta.ru/news/2019/10/28/mirumir/'

In [128]:
print('abc' + '\n\n')

abc


